In [1]:
import boa
import warnings
import json

warnings.filterwarnings("ignore")

fork_mode = True

if fork_mode:
    RPC_URL = 'https://bepolia.rpc.berachain.com/'
    boa.env.fork(RPC_URL)
else:
    boa.env.set_browser_env()
    boa.env.set_chain_id(80069)

In [2]:
# Addresses
crv_addr = '0x9eE77BFB546805fAfeB0a5e0cb150d5f82cDa47D'
crvusd_addr = '0x696FDd959BC098449aBaFF47436fcc7e1992922f' 
factory_addr = '0x714Ca028410E3cc5dD7D1F1A3b1C5A0B8d05399D'

user = '0x172dbbDa74a14268F62c87b02fC3B4438E0E328d'

In [3]:
# Load Token from Contract
token_contract_path = '../contracts/Token.vy'
token_contract = boa.load_partial(token_contract_path)

# Load factory from ABI 
factory_abi_path = 'curve-core/amm/twocryptoswap/factory/factory_v_200.json'
factory = boa.load_abi(factory_abi_path).at(factory_addr)

# TwoCrypto from ABI
pool_abi_path = 'curve-core/amm/twocryptoswap/implementation/implementation_v_210.json'
pool_contract = boa.load_abi(pool_abi_path)

# Deploy Pool

In [4]:
# https://nagaking.substack.com/p/deep-dive-curve-v2-parameters

init_price = int(0.69 * 10 ** 18)
deploy_addr = factory.deploy_pool('bera_crv_test', 
                                  'brvusd', 
                                  [crv_addr, crvusd_addr], 
                      0,  # Implementation
                      40000000, # A
                      2000000000000000,  # Gamma
                      3000000, # mid_fee
                      45000000,  #out_fee
                      300000000000000000,  #fee_gamma
                      10000000000, #allowed_extra_profit
                      5500000000000,  #adjustment_step
                      866, #ma_exp_time
                                  init_price
                                 )
pool = pool_contract.at(deploy_addr)

In [5]:
crv = token_contract.at(crv_addr)
crvusd = token_contract.at(crvusd_addr)

print(f"CRV Balance: {crv.balanceOf(user) / 10 ** 18}")
print(f"crvUSD Balance: {crv.balanceOf(user) / 10 ** 18}")
print(pool.price_oracle() / 10 ** 18)

CRV Balance: 100000000.0
crvUSD Balance: 100000000.0
0.69


# Seed Pool

In [6]:
def seed_pool(pool, amounts, crv, crvusd):
    crv.approve(pool, 2 ** 256 -1)
    crvusd.approve(pool, 2 ** 256 - 1)

    pool.add_liquidity([crv_amt, crvusd_amt], 0)
    
crvusd_amt = 10 ** 18
crv_amt = int(crvusd_amt / init_price * 10 ** 18)
amounts = [crvusd_amt, crv_amt]

if fork_mode:
    with boa.env.prank(user):
        seed_pool(pool, amounts, crv, crvusd)
else:
    seed_pool(pool, amounts, crv, crvusd)

[1000000000000000000, 1449275362318840576]
